## Run the Program

In [1]:
import cmdline_helper as cmd
import os
outdir = os.path.join("ScalarWave_Ccodes/")
grid = "72"

cmd.C_compile(os.path.join(outdir,"ScalarWave_Playground.c"), "ScalarWave_Playground")
cmd.delete_existing_files("out*.txt")
cmd.delete_existing_files("out*.png")
cmd.Execute("ScalarWave_Playground", grid, os.path.join(outdir,"out"+grid+".txt"))

Compiling executable...
Executing `gcc -Ofast -fopenmp -march=native -funroll-loops ScalarWave_Ccodes/ScalarWave_Playground.c -o ScalarWave_Playground.exe -lm`...
Finished executing in 1.8496284484863281 seconds.
Finished compilation.
Executing `cmd /c ScalarWave_Playground 72`...
Finished executing in 11.8253653049469 seconds.


## Convert text files to arrays

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import mpmath as mp
import csv

def file_reader(filename):
    with open(filename) as file:
        reader = csv.reader(file, delimiter=" ")
        data  = list(zip(*reader))
        data0 = []
        data1 = []
        data2 = []
        data3 = []
        for i in range(len(data[0])):           
            data0.append(float(data[0][i]))
            data1.append(float(data[1][i]))
            data2.append(float(data[2][i]))
            data3.append(float(data[3][i]))
        return data0,data1,data2,data3

## Plot the ScalarWave Animation in 3D

In [3]:
from mpl_toolkits import mplot3d
from matplotlib.pyplot import savefig
from matplotlib import cm

tmax = 2.4
itermax = float(grid) * tmax 
i = 0
while i <= itermax:
    filename = "out2D__resolution_"+grid+"x"+grid+"x"+grid+"__iter_"+str(i)
    x_iter_0,y_iter_0,num_0,exact_0 = file_reader(filename+".txt")
    fig = plt.figure()
    ax = plt.axes(projection='3d')
    ax.set_xlabel('x_coord')
    ax.set_ylabel('y_coord')
    ax.set_zlabel('Numerical Soln.')
    ax.set_title('ScalarWave 3D 72x72x72-resolution')
    ax.scatter3D(x_iter_0,y_iter_0,num_0, c=num_0, cmap='Greens')
#    ax.plot_trisurf(x_iter_0,y_iter_0,num_0,cmap=cm.jet, linewidth=0)
    savefig(filename+".png",dpi=450)
    plt.close(fig)
    i += 5

In [ ]:
import matplotlib.image as mgimg
from matplotlib import animation

fig = plt.figure(frameon=False)
ax = fig.add_axes([0, 0, 1, 1])
ax.axis('off')

myimages = []

i = 0
while i <= itermax:
    filename = "out2D__resolution_"+grid+"x"+grid+"x"+grid+"__iter_"+str(i)
    img = mgimg.imread(filename+".png")
    imgplot = plt.imshow(img)
    myimages.append([imgplot])
    i += 5

ani = animation.ArtistAnimation(fig, myimages, interval=100,  repeat_delay=1000)
plt.close()
ani.save("ScalarWave3D_"+grid+"_tmax_2dot4"+".mp4", fps=5,dpi=450)

## Play the Video

In [ ]:
from IPython.display import HTML
#the filename needed to be changed manually

HTML("""
    <video width="480" height="360" controls>
        <source src="ScalarWave3D_72_tmax_2dot4.mp4" type="video/mp4"> 
    </video>
""")

## Plot the ScalarWave Animation in 2D

In [9]:
i = 0
while i <= itermax:
    filename = "out2D__resolution_"+grid+"x"+grid+"x"+grid+"__iter_"+str(i)
    x_iter_0,y_iter_0,num_0,exact_0 = file_reader(filename+".txt")
    
    twoDimValue = []
    j = 0
    while j <= 50:
        array = []
        k = j * 51
        while k <= j * 51 + 50:
            array.append(num_0[k])
            k += 1
        twoDimValue.append(array)
        j += 1
    
    fig = plt.figure()
    plt.title("ScalarWave 2D 100-resolution")
    plt.xlabel("x_coord")
    plt.ylabel("y_coord")

    im = plt.imshow(twoDimValue, extent=(-5,5,-5,5))
    ax = plt.colorbar()
    ax.set_label("Numerical Soln.")
    
    savefig(filename+"_2D"+".png",dpi=450)
    plt.close(fig)
    i += 5

In [10]:
fig = plt.figure(frameon=False)
ax = fig.add_axes([0, 0, 1, 1])
ax.axis('off')

myimages = []

i = 0
while i <= itermax:
    filename = "out2D__resolution_"+grid+"x"+grid+"x"+grid+"__iter_"+str(i)
    img = mgimg.imread(filename+"_2D"+".png")
    imgplot = plt.imshow(img)
    myimages.append([imgplot])
    i += 5

ani = animation.ArtistAnimation(fig, myimages, interval=100,  repeat_delay=1000)
plt.close()
ani.save("ScalarWave2D_"+grid+".mp4", fps=5,dpi=450)

## Play the Video

In [11]:
#the filename needed to be changed manually

HTML("""
    <video width="480" height="360" controls>
        <source src="ScalarWave2D_100.mp4" type="video/mp4"> 
    </video>
""")

## Change the Initial Value

Change the ScalarWave_ExactSolution.h

const double tmp0 = time*wavespeed - (kk0*xx0 + kk1*xx1 + kk2*xx2)/sqrt(pow(kk0, 2) + pow(kk1, 2) + pow(kk2, 2));
in_gfs[IDX4(UUGF, i0, i1, i2)] = 2 - sin(tmp0);
in_gfs[IDX4(VVGF, i0, i1, i2)] = -wavespeed*cos(tmp0);

This is $$u = 2 - \sin(tmp0)$$
        $$v = - wavespeed * \cos(tmp0)$$

This is the code from ScalarWave_ExactSolution.h. Clearly, this is a sine wave with offset 2.
I leave tmp0 term unchanged.
UUGF part is the wave function. VVGF is the derivative of UUGF.

The new function is 
in_gfs[IDX4(UUGF, i0, i1, i2)] = 4 - sin(tmp0) + sin(2*tmp0) - sin(4*tmp0);
in_gfs[IDX4(VVGF, i0, i1, i2)] = -wavespeed*cos(tmp0) +2*wavespeed*cos(tmp0) -4*wavespeed*cos(tmp0);
               
This is $$u = 2 - \sin(tmp0) - \sin(2*tmp0) - \sin(4*tmp0)$$
        $$v = - wavespeed * \cos(tmp0) + 2*wavespeed * \cos(2*tmp0) - 4*wavespeed * \cos(4*tmp0)$$

In [2]:
import cmdline_helper as cmd
import os
outdir = os.path.join("ScalarWave_Ccodes/")
grid = "100"

cmd.C_compile(os.path.join(outdir,"ScalarWave_Playground_New.c"), "ScalarWave_Playground_New")
cmd.delete_existing_files("newout*.txt")
cmd.delete_existing_files("newout*.png")
cmd.Execute("ScalarWave_Playground_New", grid, os.path.join(outdir,"newout"+grid+".txt"))

Compiling executable...
Executing `gcc -Ofast -fopenmp -march=native -funroll-loops ScalarWave_Ccodes/ScalarWave_Playground_New.c -o ScalarWave_Playground_New.exe -lm`...
Finished executing in 2.660113573074341 seconds.
Finished compilation.
Executing `cmd /c ScalarWave_Playground_New 100`...
Finished executing in 60.12659168243408 seconds.


In [4]:
from mpl_toolkits import mplot3d
from matplotlib.pyplot import savefig
import matplotlib.image as mgimg
from matplotlib import animation

newtmax = 2.4
newitermax = float(grid) * newtmax
i = 0
while i <= newitermax:
    filename = "New_out2D__resolution_"+grid+"x"+grid+"x"+grid+"__iter_"+str(i)
    x_iter_0,y_iter_0,num_0,exact_0 = file_reader(filename+".txt")
    fig = plt.figure()
    ax = plt.axes(projection='3d')
    ax.set_xlabel('x_coord')
    ax.set_ylabel('y_coord')
    ax.set_zlabel('Numerical Soln.')
    ax.set_title('New ScalarWave 3D 100-resolution')

    ax.scatter3D(x_iter_0,y_iter_0,num_0, c=num_0, cmap='Reds')
    savefig(filename+".png",dpi=450)
    plt.close(fig)
    i += 5

fig = plt.figure(frameon=False)
ax = fig.add_axes([0, 0, 1, 1])
ax.axis('off')

myimages = []

i = 0
while i <= newitermax:
    filename = "New_out2D__resolution_"+grid+"x"+grid+"x"+grid+"__iter_"+str(i)
    img = mgimg.imread(filename+".png")
    imgplot = plt.imshow(img)
    myimages.append([imgplot])
    i += 5

ani = animation.ArtistAnimation(fig, myimages, interval=100,  repeat_delay=1000)
plt.close()
ani.save("New_ScalarWave3D_"+grid+".mp4", fps=5,dpi=450)

In [5]:
from IPython.display import HTML
#the filename needed to be changed manually

HTML("""
    <video width="480" height="360" controls>
        <source src="New_ScalarWave3D_100.mp4" type="video/mp4"> 
    </video>
""")

In [6]:
i = 0
while i <= newitermax:
    filename = "New_out2D__resolution_"+grid+"x"+grid+"x"+grid+"__iter_"+str(i)
    x_iter_0,y_iter_0,num_0,exact_0 = file_reader(filename+".txt")
    
    twoDimValue = []
    j = 0
    while j <= 50:
        array = []
        k = j * 51
        while k <= j * 51 + 50:
            array.append(num_0[k])
            k += 1
        twoDimValue.append(array)
        j += 1
    
    fig = plt.figure()
    plt.title("New ScalarWave 2D 100-resolution")
    plt.xlabel("x_coord")
    plt.ylabel("y_coord")

    im = plt.imshow(twoDimValue, extent=(-5,5,-5,5))
    ax = plt.colorbar()
    ax.set_label("Numerical Soln.")
    
    savefig(filename+"_2D"+".png",dpi=450)
    plt.close(fig)
    i += 5

fig = plt.figure(frameon=False)
ax = fig.add_axes([0, 0, 1, 1])
ax.axis('off')

myimages = []

i = 0
while i <= newitermax:
    filename = "New_out2D__resolution_"+grid+"x"+grid+"x"+grid+"__iter_"+str(i)
    img = mgimg.imread(filename+"_2D"+".png")
    imgplot = plt.imshow(img)
    myimages.append([imgplot])
    i += 5

ValueError: list.remove(x): x not in list

<Figure size 432x288 with 1 Axes>

In [7]:
ani = animation.ArtistAnimation(fig, myimages, interval=100,  repeat_delay=1000)
plt.close()
ani.save("New_ScalarWave2D_"+grid+".mp4", fps=5,dpi=450)

In [8]:
#the filename needed to be changed manually

HTML("""
    <video width="480" height="360" controls>
        <source src="New_ScalarWave2D_100.mp4" type="video/mp4"> 
    </video>
""")

## The Exact Solution for this function

In [10]:
newtmax = 2.4
newitermax = float(grid) * newtmax
i = 0
while i <= newitermax:
    filename = "New_out2D__resolution_"+grid+"x"+grid+"x"+grid+"__iter_"+str(i)
    x_iter_0,y_iter_0,num_0,exact_0 = file_reader(filename+".txt")
    fig = plt.figure()
    ax = plt.axes(projection='3d')
    ax.set_xlabel('x_coord')
    ax.set_ylabel('y_coord')
    ax.set_zlabel('Exact Soln.')
    ax.set_title('New ScalarWave 3D 100-resolution (Exact)')

    ax.scatter3D(x_iter_0,y_iter_0,exact_0, c=exact_0, cmap='Reds')
    savefig('Exact_'+filename+".png",dpi=450)
    plt.close(fig)
    i += 5

fig = plt.figure(frameon=False)
ax = fig.add_axes([0, 0, 1, 1])
ax.axis('off')

myimages = []

i = 0
while i <= newitermax:
    filename = "Exact_New_out2D__resolution_"+grid+"x"+grid+"x"+grid+"__iter_"+str(i)
    img = mgimg.imread(filename+".png")
    imgplot = plt.imshow(img)
    myimages.append([imgplot])
    i += 5

ani = animation.ArtistAnimation(fig, myimages, interval=100,  repeat_delay=1000)
plt.close()
ani.save("Exact_New_ScalarWave3D_"+grid+".mp4", fps=5,dpi=450)

In [12]:
#the filename needed to be changed manually

HTML("""
    <video width="480" height="360" controls>
        <source src="Exact_New_ScalarWave3D_100.mp4" type="video/mp4"> 
    </video>
""")

## Change the Boundary Condition

In [1]:
import cmdline_helper as cmd
import os
outdir = os.path.join("ScalarWave_Ccodes/")
grid = "50"

cmd.C_compile(os.path.join(outdir,"ScalarWave_Playground_experiment.c"), "ScalarWave_Playground_experiment")
cmd.delete_existing_files("newboundout*.txt")
cmd.delete_existing_files("newboundout*.png")
cmd.Execute("ScalarWave_Playground_experiment", grid, os.path.join(outdir,"newboundout"+grid+".txt"))

Compiling executable...
Executing `gcc -Ofast -fopenmp -march=native -funroll-loops ScalarWave_Ccodes/ScalarWave_Playground_experiment.c -o ScalarWave_Playground_experiment.exe -lm`...
ScalarWave_Ccodes/ScalarWave_Playground_experiment.c: In function 'output_2D':
ScalarWave_Ccodes/ScalarWave_Playground_experiment.c:123:60: error: 'i0' undeclared (first use in this function)
                         numerical_gridfunction_data[IDX4(0,i0,i1, (int)((Nxx[2]+ 2*NGHOSTS)*0.5))],
                                                            ^
ScalarWave_Ccodes/ScalarWave_Playground_experiment.c:21:4: note: in definition of macro 'IDX4'
 ( (i) + Nxx_plus_2NGHOSTS[0] * ( (j) + Nxx_plus_2NGHOSTS[1] * ( (k) + Nxx_plus_2NGHOSTS[2] * (g) ) ) )
    ^
ScalarWave_Ccodes/ScalarWave_Playground_experiment.c:123:60: note: each undeclared identifier is reported only once for each function it appears in
                         numerical_gridfunction_data[IDX4(0,i0,i1, (int)((Nxx[2]+ 2*NGHOSTS)*0.5))],
      

SystemExit: 1

D:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3333: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
from mpl_toolkits import mplot3d
from matplotlib.pyplot import savefig

tmax = 2.4
itermax = float(grid) * tmax 
i = 0
while i <= itermax:
    filename = "out2D__resolution_"+grid+"x"+grid+"x"+grid+"__iter_"+str(i)
    x_iter_0,y_iter_0,num_0,exact_0 = file_reader(filename+".txt")
    fig = plt.figure()
    ax = plt.axes(projection='3d')
    ax.set_xlabel('x_coord')
    ax.set_ylabel('y_coord')
    ax.set_zlabel('Numerical Soln.')
    ax.set_title('ScalarWave 3D 50-resolution with new Boundary')

    ax.scatter3D(x_iter_0,y_iter_0,num_0, c=num_0, cmap='Greens')
    savefig(filename+".png",dpi=450)
    plt.close(fig)
    i += 5

In [ ]:
import matplotlib.image as mgimg
from matplotlib import animation

fig = plt.figure(frameon=False)
ax = fig.add_axes([0, 0, 1, 1])
ax.axis('off')

myimages = []

i = 0
while i <= itermax:
    filename = "out2D__resolution_"+grid+"x"+grid+"x"+grid+"__iter_"+str(i)
    img = mgimg.imread(filename+".png")
    imgplot = plt.imshow(img)
    myimages.append([imgplot])
    i += 5

ani = animation.ArtistAnimation(fig, myimages, interval=100,  repeat_delay=1000)
plt.close()
ani.save("New_Boundary_ScalarWave3D_"+grid+".mp4", fps=5,dpi=450)

In [ ]:
from IPython.display import HTML
#the filename needed to be changed manually

HTML("""
    <video width="480" height="360" controls>
        <source src="New_Boundary_ScalarWave3D_50.mp4" type="video/mp4"> 
    </video>
""")

In [ ]:
from IPython.display import HTML
#the filename needed to be changed manually

HTML("""
    <video width="480" height="360" controls>
        <source src="New_Boundary_ScalarWave3D_50_tmax_2dot4.mp4" type="video/mp4"> 
    </video>
""")

In [14]:
i = 0
while i <= itermax:
    filename = "out2D__resolution_"+grid+"x"+grid+"x"+grid+"__iter_"+str(i)
    x_iter_0,y_iter_0,num_0,exact_0 = file_reader(filename+".txt")
    
    twoDimValue = []
    j = 0
    while j <= 50:
        array = []
        k = j * 51
        while k <= j * 51 + 50:
            array.append(num_0[k])
            k += 1
        twoDimValue.append(array)
        j += 1
    
    fig = plt.figure()
    plt.title("ScalarWave 2D 50-resolution with new boundary")
    plt.xlabel("x_coord")
    plt.ylabel("y_coord")

    im = plt.imshow(twoDimValue, extent=(-10,10,-10,10))
    ax = plt.colorbar()
    ax.set_label("Numerical Soln.")
    
    savefig(filename+"_2D"+".png",dpi=450)
    plt.close(fig)
    i += 5

In [15]:
fig = plt.figure(frameon=False)
ax = fig.add_axes([0, 0, 1, 1])
ax.axis('off')

myimages = []

i = 0
while i <= itermax:
    filename = "out2D__resolution_"+grid+"x"+grid+"x"+grid+"__iter_"+str(i)
    img = mgimg.imread(filename+"_2D"+".png")
    imgplot = plt.imshow(img)
    myimages.append([imgplot])
    i += 5

ani = animation.ArtistAnimation(fig, myimages, interval=100,  repeat_delay=1000)
plt.close()
ani.save("New Boundary2 ScalarWave2D_"+grid+".mp4", fps=5,dpi=450)

In [37]:
#the filename needed to be changed manually

HTML("""
    <video width="480" height="360" controls>
        <source src="New Boundary ScalarWave2D_50.mp4" type="video/mp4"> 
    </video>
""")

In [16]:
#the filename needed to be changed manually

HTML("""
    <video width="480" height="360" controls>
        <source src="New Boundary2 ScalarWave2D_50.mp4" type="video/mp4"> 
    </video>
""")